<a href="https://colab.research.google.com/github/BaikaiL/voice/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 安装 Edge-TTS (用于高质量语音合成)
!pip install edge-tts nest_asyncio

In [11]:
import torch
import asyncio
import nest_asyncio
import edge_tts
from transformers import pipeline, AutoProcessor, AutoModelForSpeechSeq2Seq # 引入更底层的类
import IPython.display as ipd
import uuid
import gradio as gr

# 解决 Jupyter/Colab 中的异步循环冲突问题
nest_asyncio.apply()

class CantonesToMandarinTranslator:
    def __init__(self):
        # 自动检测 GPU
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

        print(f"正在初始化... (Device: {self.device})")

        # 1. 加载 ASR 模型
        print("正在加载 Whisper (ASR)...")

        # 1. 从仓库加载微调后的模型权重
        my_model_id = "baikai1022/whisper-small-cantonese-v2"
        model = AutoModelForSpeechSeq2Seq.from_pretrained(
            my_model_id,
            torch_dtype=self.torch_dtype,
            low_cpu_mem_usage=True,
            use_safetensors=True
        ).to(self.device)

        # 2. 从 OpenAI 原版加载处理器/分词器 (填补仓库缺失的文件)
        processor = AutoProcessor.from_pretrained("openai/whisper-small")

        # 3. 手动组装 Pipeline
        self.asr_pipe = pipeline(
            "automatic-speech-recognition",
            model=model,
            tokenizer=processor.tokenizer,           # 借用原版 tokenizer
            feature_extractor=processor.feature_extractor, # 借用原版 feature_extractor
            device=self.device,
            torch_dtype=self.torch_dtype,
        )

         # 2. 加载 NLLB 模型 (Translation)
        print("正在加载 NLLB (Translation)...")
        self.mt_pipe = pipeline(
            "translation",
            model="baikai1022/nllb-cantonese-mandarin-v1",
            device=self.device,
            torch_dtype=self.torch_dtype,
            src_lang="yue_Hant",
            tgt_lang="zho_Hans"
        )
        print("模型加载完成！")

    def run(self, audio_path, output_audio_path="output_mandarin.mp3"):
        print("\n" + "="*40)
        print(f"处理音频: {audio_path}")

        # --- 步骤 1: 语音识别 ---
        asr_result = self.asr_pipe(
            audio_path,
        )
        cantonese_text = asr_result["text"]
        print(f"[识别结果 (粤语)]: {cantonese_text}")

        # --- 步骤 2: 机器翻译 ---
        mt_result = self.mt_pipe(cantonese_text)
        mandarin_text = mt_result[0]['translation_text']
        print(f"[翻译结果 (国语)]: {mandarin_text}")

        # --- 步骤 3: 语音合成 ---
        print("正在合成语音...")
        voice = "zh-CN-XiaoxiaoNeural"

        async def _generate_tts():
            communicate = edge_tts.Communicate(mandarin_text, voice)
            await communicate.save(output_audio_path)

        asyncio.run(_generate_tts())

        print(f"合成完成: {output_audio_path}")
        print("="*40 + "\n")
        return output_audio_path, mandarin_text

    def process(self, audio_path):
        """
        专门给 Gradio 用的处理函数
        返回: (粤语文本, 普通话文本, 普通话音频路径)
        """
        if not audio_path:
            return "请先录音或上传文件", "", None

        print(f"正在处理: {audio_path}")

        # 1. ASR 识别
        # 指定 language="chinese" 以利用微调效果且避免报错
        asr_result = self.asr_pipe(audio_path)
        cantonese_text = asr_result["text"]

        # 2. MT 翻译
        mt_result = self.mt_pipe(cantonese_text)
        mandarin_text = mt_result[0]['translation_text']

        # 3. TTS 合成
        # 使用随机文件名，防止多人使用时冲突
        output_filename = f"out_{uuid.uuid4().hex[:8]}.mp3"
        voice = "zh-CN-XiaoxiaoNeural"

        async def _gen_tts():
            comm = edge_tts.Communicate(mandarin_text, voice)
            await comm.save(output_filename)

        asyncio.run(_gen_tts())

        return cantonese_text, mandarin_text, output_filename

# 运行测试
translator = CantonesToMandarinTranslator()



正在初始化... (Device: cuda)
正在加载 Whisper (ASR)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda


正在加载 NLLB (Translation)...


Device set to use cuda


模型加载完成！


In [12]:
# 请确保这里有文件，或者修改文件名
input_audio = "common_voice_zh-HK_20110217.wav"

import os
if os.path.exists(input_audio):
    output, text = translator.run(input_audio)
    ipd.display(ipd.Audio(output))
else:
    print(f"请上传一个名为 {input_audio} 的粤语录音文件来测试")


处理音频: common_voice_zh-HK_20110217.wav


`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
`generation_config` default values have been modified to match model-specific defaults: {'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536

[识别结果 (粤语)]: 有个老婆婆喺大埔教翠联路等緊小巴。
[翻译结果 (国语)]: 有个老婆婆在大埔教联路等小巴。
正在合成语音...
合成完成: output_mandarin.mp3



In [15]:
def gradio_interface(audio):
    return translator.process(audio)

with gr.Blocks(title="粤语转普通话 AI") as demo:
    gr.Markdown("# 🇭🇰 粤语 -> 🇨🇳 普通话 智能转换器")
    gr.Markdown("基于 Whisper-Small (微调) + NLLB-200 (微调) + Edge-TTS")

    with gr.Row():
        # 左侧：输入区
        with gr.Column():
            # sources=["microphone"] 允许录音，type="filepath" 传递文件路径
            input_audio = gr.Audio(sources=["microphone", "upload"], type="filepath", label="请说粤语")
            btn = gr.Button("开始转换", variant="primary")

        # 右侧：输出区
        with gr.Column():
            txt_cantonese = gr.Textbox(label="识别结果 (粤语)")
            txt_mandarin = gr.Textbox(label="翻译结果 (普通话)")
            out_audio = gr.Audio(label="合成语音 (普通话)", autoplay=True)

    # 绑定点击事件
    btn.click(
        fn=gradio_interface,
        inputs=input_audio,
        outputs=[txt_cantonese, txt_mandarin, out_audio]
    )
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7dc314bbb3990f714f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在处理: /tmp/gradio/38471469a1b23996af3423040de9c0628dd103670361c65df7fa5314aa8b2f78/audio.wav
正在处理: /tmp/gradio/501261b72385edd2c10d5872f8729f948d87f98f3a39753012fffbce7878411d/audio.wav


Your input_length: 333 is bigger than 0.9 * max_length: 200. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7dc314bbb3990f714f.gradio.live


In [25]:
pip install pyecharts

In [26]:
pip install plotly

In [13]:
import gradio as gr
import psutil
import torch
import plotly.graph_objects as go  # 导入 Plotly

# --- 1. 创建 Plotly 绘图函数 ---
def create_perf_plot(ram_value, vram_value):
    # 创建一个交互式柱状图
    fig = go.Figure(data=[
        go.Bar(
            x=["系统内存 (RAM)", "显存峰值 (VRAM)"],
            y=[round(ram_value, 2), round(vram_value, 2)],
            marker_color=['#636EFA', '#EF553B'], # 设置蓝色和红色
            text=[f"{ram_value:.2f} GB", f"{vram_value:.2f} GB"], # 柱子上显示的文字
            textposition='auto',
        )
    ])

    fig.update_layout(
        title="硬件资源占用统计 (单位: GB)",
        yaxis=dict(range=[0, 16]), # 根据 Colab 常见内存上限设置坐标轴范围
        height=300,
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="rgba(0,0,0,0)", # 透明背景
        plot_bgcolor="rgba(0,0,0,0)"
    )
    return fig

# --- 2. 接口函数 ---
def gradio_interface(audio):
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()

    # 执行你原本的翻译逻辑
    # 假设 translator.process 返回: 粤语文本, 普通话文本, 音频
    cantonese_res, mandarin_res, audio_res = translator.process(audio)

    # 获取资源数据
    ram = psutil.virtual_memory().used / (1024**3)
    vram = 0
    if torch.cuda.is_available():
        vram = torch.cuda.max_memory_allocated() / (1024**3)

    # 生成 Plotly 图表对象
    fig = create_perf_plot(ram, vram)

    return cantonese_res, mandarin_res, audio_res, fig

# --- 3. 构建 UI 界面 ---
with gr.Blocks(title="粤语转普通话 AI") as demo:
    gr.Markdown("# 🇭🇰 粤语 -> 🇨🇳 普通话 智能转换器")

    with gr.Row():
        # 左侧输入
        with gr.Column():
            input_audio = gr.Audio(sources=["microphone", "upload"], type="filepath", label="请说粤语")
            btn = gr.Button("🚀 开始转换", variant="primary")

            # --- 重要：使用 gr.Plot 专门显示交互式图表 ---
            perf_plot = gr.Plot(label="资源占用监控")

        # 右侧输出
        with gr.Column():
            txt_cantonese = gr.Textbox(label="识别结果 (粤语)")
            txt_mandarin = gr.Textbox(label="翻译结果 (普通话)")
            out_audio = gr.Audio(label="合成语音 (普通话)", autoplay=True)

    # 绑定事件
    btn.click(
        fn=gradio_interface,
        inputs=input_audio,
        outputs=[txt_cantonese, txt_mandarin, out_audio, perf_plot]
    )

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f10f3a5454acd90d56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


正在处理: /tmp/gradio/3489f36f5fe91be962bf7dd6e3d17b1d1e3678a1fc3038ff85702bff7fb7a7e4/audio.wav
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://f10f3a5454acd90d56.gradio.live


In [14]:
!pip install --upgrade gradio plotly pandas
# 运行后如果提示 "Restart Session"，请点击按钮重启，否则代码会报错。

In [ ]:
import gradio as gr
import psutil
import torch
import pandas as pd
import time
import plotly.express as px
from datetime import datetime

# --- 1. 监控数据与绘图逻辑 ---
def get_perf_figure(history_df):
    """根据数据生成 Plotly 图表"""
    if history_df.empty:
        # 初始空图表，防止白屏
        fig = px.line(title="性能监控启动中...")
    else:
        fig = px.line(
            history_df, x="Time", y="Value", color="Type",
            title="实时性能监控 (秒级更新)",
            range_y=[0, 8],
            template="plotly_white"
        )
    fig.update_layout(height=300, margin=dict(l=10, r=10, t=40, b=10))
    return fig

# 存储历史数据
if 'perf_history' not in globals():
    perf_history = pd.DataFrame(columns=["Time", "Value", "Type"])

def update_monitor_plot():
    global perf_history
    now = datetime.now().strftime("%H:%M:%S")

    # 采集数据
    ram = psutil.virtual_memory().used / (1024**3)
    vram = 0
    if torch.cuda.is_available():
        vram = torch.cuda.memory_allocated() / (1024**3)

    # 更新数据
    new_data = pd.DataFrame([
        {"Time": now, "Value": ram, "Type": "系统内存 (GB)"},
        {"Time": now, "Value": vram, "Type": "显存 (GB)"}
    ])
    perf_history = pd.concat([perf_history, new_data], ignore_index=True).tail(60)
    return get_perf_figure(perf_history)

# --- 2. 翻译接口 ---
def gradio_interface(audio):
    return translator.process(audio)

# --- 3. UI 界面 ---
with gr.Blocks(title="粤语转普通话 AI") as demo:
    gr.Markdown("# 🇭🇰 粤语 -> 🇨🇳 普通话 (性能实时监控版)")

    with gr.Row():
        with gr.Column():
            input_audio = gr.Audio(sources=["microphone", "upload"], type="filepath", label="请说粤语")
            btn = gr.Button("开始转换", variant="primary")

            # 性能折线图 - 给予初始图表防止加载时空白
            perf_plot = gr.Plot(value=get_perf_figure(perf_history), label="性能趋势")

            # --- 调试点：先把刷新按钮设为可见，如果你点它图表动了，说明是 JS 没生效 ---
            hidden_btn = gr.Button("手动刷新监控 (测试用)", elem_id="refresh_trigger", visible=True)

        with gr.Column():
            txt_cantonese = gr.Textbox(label="识别结果 (粤语)")
            txt_mandarin = gr.Textbox(label="翻译结果 (普通话)")
            out_audio = gr.Audio(label="合成语音 (普通话)", autoplay=True)

    # 绑定翻译
    btn.click(fn=gradio_interface, inputs=input_audio, outputs=[txt_cantonese, txt_mandarin, out_audio])

    # 绑定监控
    hidden_btn.click(fn=update_monitor_plot, inputs=None, outputs=perf_plot)

    # --- 增强版 JavaScript 注入 ---
    gr.HTML("""
        <script>
            function startAutoClicker() {
                console.log("Monitoring script started...");
                setInterval(function() {
                    // 尝试通过多种方式查找按钮
                    var btn = document.getElementById('refresh_trigger') ||
                              document.querySelector('#refresh_trigger') ||
                              document.querySelector('button.secondary'); // 备选方案

                    if (btn) {
                        // console.log("Clicking refresh..."); // 调试用
                        btn.click();
                    }
                }, 1000);
            }
            // 延迟 3 秒启动，确保界面完全加载
            setTimeout(startAutoClicker, 3000);
        </script>
    """)

# 启动
demo.launch(share=True, debug=True)